### Simulação Post-Predict

In [1]:
import findspark
findspark.init('/home/labdata/spark-2.2.1-bin-hadoop2.6')

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession\
            .builder\
            .appName('bank')\
            .getOrCreate()

In [3]:
from pyspark.sql import functions as F

In [4]:
model_output = spark.read.csv(
    "hdfs://elephant:8020/user/labdata/predictions/2018-08-09/predictions/part-00000-ab65832c-2463-4ecc-bf77-a46ade62715f-c000.csv",
    header=False,
    sep=",",
    inferSchema=True
)

colnames = ['client_id', 'prediction', 'label', 'score']
model_output = model_output.selectExpr(*["{} as {}".format(model_output.columns[i], colnames[i]) for i in range(len(model_output.columns))])

. Qual a taxa de conversão para os grupos que o modelo previu compra vs não-compra?

In [5]:
results = model_output.groupBy('prediction').agg({"*": "count", "label": "avg"})
results = results.select(
    F.when(results['prediction']==1, 'yes').otherwise('no').alias('prediction'), 
    (results['count(1)']).alias('count'), F
    .format_number('avg(label)', 4).alias('convertion_rate')
)
results.show()

+----------+-----+---------------+
|prediction|count|convertion_rate|
+----------+-----+---------------+
|        no| 6876|         0.0558|
|       yes| 1361|         0.3990|
+----------+-----+---------------+



. Digamos que só temos mão para realizar 500 ligações (número arbitrário para fins de ilustração), quão melhor é a conversão se utilizando o modelo vs. escolher aleatóriamente? 

In [6]:
# aleatoriamente
random_sample = model_output.sample(
    withReplacement=False,
    fraction=0.0623,
    seed=420
)
cr_rand = 100*random_sample.groupBy().avg('label').collect()[0][0]

In [7]:
# top500
top500_sample = model_output.orderBy(F.desc('score')).limit(500)
cr_top = 100*top500_sample.groupBy().avg('label').collect()[0][0]

In [8]:
print(f"Taxa de Conversão Esperada:\n\tSem Modelo: {cr_rand:.2f}%\n\tCom Modelo: {cr_top:.2f}%")

Taxa de Conversão Esperada:
	Sem Modelo: 10.78%
	Com Modelo: 56.60%


Um aumento significativo em efetividade!